## 3584번: 가장 가까운 공통 조상

In [13]:
import sys
from collections import deque
#input = sys.stdin.readline

def LCA(a, b):
    if depth[a] > depth[b]:
        a, b = b, a
    while depth[a] != depth[b]:
        b = before[b]
    while a != b:
        a = before[a]
        b = before[b]
    return a

def find_root(graph):
    count = [0] * (N+1)
    count[0] = 1
    for i in graph:
        for j in i:
            count[j] = 1
    return count.index(0)

T = int(input())
for t in range(T):
    N = int(input())
    graph = [[] for i in range(N+1)]
    for _ in range(N-1):
        A, B = map(int, input().split())
        graph[A].append(B)

    depth = [0] * (N+1)
    before = [0] * (N+1)
    root = find_root(graph)
    queue = deque()
    queue.append(root)
    while queue:
        select = queue.popleft()
        for n in graph[select]:
            if depth[n] == 0:
                queue.append(n)
                depth[n] = depth[select] + 1
                before[n] = select
    A, B = map(int, input().split())
    print(LCA(A, B))

2
16
1 14
8 5
10 16
5 9
4 6
8 4
4 10
1 13
6 15
10 11
6 7
10 2
16 3
8 1
16 12
16 7
4
5
2 3
3 4
3 1
1 5
3 5
3


## 17435번: 합성함수와 쿼리

In [23]:
import sys
#input = sys.stdin.readline
m = int(input())
f = [[0] * (m+1) for i in range(20)]
f[0] = [0] + list(map(int, input().split()))

for i in range(1, 20):
    for j in range(1, m+1):
        f[i][j] = f[i-1][f[i-1][j]]
        
q =  int(input())
for _ in range(q):
    n, x = map(int, input().split())
    for i in range(18, -1, -1):
        if n >= 2**i:
            n -= 2**i
            x = f[i][x]
    print(x)

5
3 3 5 4 3
5
1 1
3
2 1
5
11 3
5
1000 4
4
5 1
3


## 11438번: LCA 2

In [11]:
import sys
#input = sys.stdin.readline
sys.setrecursionlimit(100000) #재귀 깊이 미설정시 RecursionError 발생

def DFS(x, d):
    visited[x] = 1
    depth[x] = d
    for n in graph[x]:
        if visited[n] != 1:
            parents[n][0] = x
            DFS(n, d+1)

def sparse_table():
    for j in range(1, 21):
        for i in range(1, N+1):
            parents[i][j] = parents[parents[i][j-1]][j-1]
            
def LCA(x, y):
    if depth[x]>depth[y]:
        x, y = y, x
    for i in range(20, -1, -1):
        if(depth[y]-depth[x]>=(1<<i)):
            y=parents[y][i]     
    #print(x, y)
    if x==y:
        return x
    for i in range(20, -1, -1):
        if parents[x][i]!=parents[y][i]:
            x = parents[x][i]
            y = parents[y][i]
    return parents[x][0]

N = int(input())
graph = [[] for i in range(N+1)]
for _ in range(N-1):
    A, B = map(int, input().split())
    graph[A].append(B)
    graph[B].append(A)
    
depth = [0] * (N+1)
root = 1
visited = [0] * (N+1)
parents = [[0]*21 for i in range(N+1)]
DFS(root, 0)

sparse_table()
M = int(input())
for _ in range(M):
    A, B = map(int, input().split())
    print(LCA(A, B))

15
1 2
1 3
2 4
3 7
6 2
3 8
4 9
2 5
5 11
7 13
10 4
11 15
12 5
14 7
6
6 11
2
10 9
4
2 6
2
7 6
1
8 13
3
8 15
1


## 3176번: 도로 네트워크 (미완성)

In [51]:
import sys
#input = sys.stdin.readline

N = int(input())
graph = [[] for i in range(N+1)]
for _ in range(N-1):
    A, B, C = map(int, input().split())
    graph[A].append([B, C])
    graph[B].append([A, C])

6
2 3 100
4 3 200
1 5 150
1 3 50
6 2 250


In [59]:
from collections import deque

visited = [0] * (N+1)
depth = [0] * (N+1)
parents = [[0]*21 for i in range(N+1)]
max_length = [[0]*21 for i in range(N+1)]
min_length = [[0]*21 for i in range(N+1)]

def BFS(root):
    visited[root] = 1
    queue = deque()
    queue.append(root)
    while queue:
        select = queue.popleft()
        for i, length in graph[select]:
            if visited[i] == 0:
                parents[i][0] = select
                min_length[i][0] = length
                max_length[i][0] = length
                queue.append(i)
                depth[i] = depth[select] + 1
                visited[i] = 1

def sparse_table():
    global max_length, min_length
    for j in range(1, 21):
        for i in range(1, N+1):
            parents[i][j] = parents[parents[i][j-1]][j-1]
            #print(min_length[i][j-1], min_length[parents[i][j-1]][j-1])
            min_length[i][j] = min(min_length[i][j-1], min_length[parents[i][j-1]][j-1])
            max_length[i][j] = max(max_length[i][j-1], max_length[parents[i][j-1]][j-1])
            
INF = 1000000
def LCA(x, y):
    if x==y:
        return 0, 0

    if depth[x]>depth[y]:
        x, y = y, x
        
    min_value = min_length[y][0]
    max_value = max_length[y][0]

    if parents[y][0] == x:
        return min_length[y][0], max_length[y][0]
    
    for i in range(20, -1, -1):
        if(depth[y]-depth[x]>=(1<<i)):
            min_value = min(min_value, min_length[y][i])
            max_value = max(max_value, max_length[y][i])
            y = parents[y][i]
    #print(x, y)
    if x==y:
        return min_value, max_value
    
    for i in range(20, -1, -1):
        if parents[x][i]!=parents[y][i]:
            print(i)
            min_value = min([min_value, min_length[x][i], min_length[y][i]])
            max_value = max([max_value, max_length[x][i], max_length[y][i]])
            x = parents[x][i]
            y = parents[y][i]
    min_value = min([min_value, min_length[x][i], min_length[y][i]])
    max_value = max([max_value, max_length[x][i], max_length[y][i]])
    return min_value, max_value

In [64]:
BFS(1)
sparse_table()
K = int(input())
for _ in range(K):
    D, E = map(int, input().split())
    min_value, max_value = LCA(D, E)
    print(min_value, max_value)

1
2 5
50 150
